<center>
<h1>Stone Soup Tracker Constructor Demo</h1>
Building a tracker from a Stone Soup configuration file, including varying input parameters. To run this notebook the 'runmanager' Stone Soup branch must first be checked out.
</center>

Set up initial import.

In [1]:
import datetime
import numpy as np
import copy as copy
import tempfile
import itertools

We must import all the objects that we will build our tracker with.

In [2]:
from collections import defaultdict
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.simulator.simple import MultiTargetGroundTruthSimulator
from stonesoup.simulator.simple import SimpleDetectionSimulator
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.models.transition.linear import ConstantVelocity
from stonesoup.types.state import GaussianState
from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.hypothesiser.distance import MahalanobisDistanceHypothesiser
from stonesoup.dataassociator.neighbour import NearestNeighbour
from stonesoup.initiator.simple import SinglePointInitiator
from stonesoup.deleter.error import CovarianceBasedDeleter
from stonesoup.tracker.simple import MultiTargetTracker
from stonesoup.serialise import YAML
from stonesoup.reader.config import ConfigConstituter

Now we build the components of our tracker to save in the config file. This process will be replaced with a configuration constructor. The components are built as a set of dictionaries containing 'instructions' on how to build the tracker.

In [3]:
config_transition_model = {'class': 'CombinedLinearGaussianTransitionModel',
                           'model_list':(ConstantVelocity(1), ConstantVelocity(1))}

In [4]:
config_measurement_model = {'class': 'LinearGaussian',
                            'ndim_state': 4,
                            'mapping':[0, 2],
                            'noise_covar':np.diag([10, 50])}

In [5]:
config_groundtruth_initial_state = {'class': 'GaussianState',
                                    'state_vector': StateVector([[0], [0], [0], [0]]),
                                    'covar': CovarianceMatrix(np.diag([1000000, 1000, 1000000, 1000]))
                                   }

config_groundtruth_source = {'class': 'MultiTargetGroundTruthSimulator',
                             'transition_model': config_transition_model,
                             'initial_state': config_groundtruth_initial_state,
                             'timestep': datetime.timedelta(seconds=1),
                             'birth_rate': 0.3,
                             'death_probability': 0.05
                            }

In [6]:
config_detection_source = {'class': 'SimpleDetectionSimulator',
                           'groundtruth': config_groundtruth_source,
                            'measurement_model': config_measurement_model,
                            'meas_range': np.array([[-1, 1], [-1, 1]])*5000,
                            'detection_probability': 0.8,
                            'clutter_rate': 5
                           }

In [7]:
config_transition_model_01 = {'class': 'CombinedLinearGaussianTransitionModel',
                              'model_list': (ConstantVelocity(1), ConstantVelocity(3)),
                             }
config_transition_model_02 = {'class': 'CombinedLinearGaussianTransitionModel',
                              'model_list': (ConstantVelocity(2), ConstantVelocity(4)),
                             }
config_transition_models = [config_transition_model_01, config_transition_model_02]

config_predictor = {'class': 'KalmanPredictor',
                    'transition_model': {'options': config_transition_models}
                   }

In [8]:
config_updater = {'class': 'KalmanUpdater',
                  'measurement_model': config_measurement_model
                 }

In [9]:
config_hypothesiser = {'class': 'MahalanobisDistanceHypothesiser',
                        'predictor': config_predictor,
                        'updater': config_updater
                       }

config_data_associator = {'class': 'NearestNeighbour',
                          'hypothesiser': config_hypothesiser
                         }

In [10]:
config_initiator = {'class': 'SinglePointInitiator',
                    'prior_state': GaussianState(np.array([[0], [0], [0], [0]]), np.diag([10000, 100, 10000, 1000])),
                    'measurement_model': config_measurement_model
                   }

In [11]:
config_deleter = {'class': 'CovarianceBasedDeleter',
                  'covar_trace_thresh': 0.5e3
                 }

In [12]:
config_tracker = {'class': 'MultiTargetTracker',
                  'initiator': config_initiator,
                  'deleter': config_deleter,
                  'detector': config_detection_source,
                  'data_associator': config_data_associator,
                  'updater': config_updater
                 }

Next we specify a set of conditions with which to modify the tracker. In this case we are setting a number of repeat runs and changing variables within the tracker.

In [13]:
repeat_condition = {'num_iter': 2}
range_condition_1 = {'var': 'covar_trace_thresh', 'values': [0.2e3, 1e3]}
range_condition_2 = {'var': 'clutter_rate', 'values': [2, 8]}

config_conditions = {'REPEAT': repeat_condition, 'RANGE': [range_condition_1, range_condition_2]}

Finally, we build the configuration file by combining the tracker and the modification conditions.

In [14]:
config_top_level = {}
config_top_level['tracker'] = config_tracker
config_top_level['conditions'] = config_conditions

In [15]:
config_file = YAML().dumps(config_top_level)

print(config_file)

tracker:
  class: MultiTargetTracker
  initiator:
    class: SinglePointInitiator
    prior_state: !stonesoup.types.state.GaussianState
    - state_vector: !numpy.ndarray
      - [0]
      - [0]
      - [0]
      - [0]
    - covar: !numpy.ndarray
      - [10000, 0, 0, 0]
      - [0, 100, 0, 0]
      - [0, 0, 10000, 0]
      - [0, 0, 0, 1000]
    measurement_model: &id001
      class: LinearGaussian
      ndim_state: 4
      mapping:
      - 0
      - 2
      noise_covar: !numpy.ndarray
      - [10, 0]
      - [0, 50]
  deleter:
    class: CovarianceBasedDeleter
    covar_trace_thresh: 500.0
  detector:
    class: SimpleDetectionSimulator
    groundtruth:
      class: MultiTargetGroundTruthSimulator
      transition_model:
        class: CombinedLinearGaussianTransitionModel
        model_list:
        - !stonesoup.models.transition.linear.ConstantVelocity
          - noise_diff_coeff: 1
        - !stonesoup.models.transition.linear.ConstantVelocity
          - noise_diff_coeff: 1
     

We will save this config to load in later.

In [16]:
file = tempfile.NamedTemporaryFile(mode='w', prefix="soup_config_", delete=False)
file.write(config_file)
file.close()
print(file.name)

C:\Users\CJENGL~1\AppData\Local\Temp\soup_config_remo51z0


Now we read in our saved configuration file. By setting the yaml typ arument to 'safe' we bring in the config as a dictionary rather than a yaml-specific type.

In [17]:
with open(file.name, 'r') as myfile:
    config = YAML(typ='safe').load(myfile.read())

The tracker constructor is a subclass of Stone Soup reader. It builds a tracker by recursively calling a function to unpack the nested sets of dictionaries.

In [18]:
constituter = ConfigConstituter()

tracker = constituter.decode_config(config)

Now all that is left is to run the tracker in it's various configurations specified in the config.

In [19]:
total_groundtruth_paths = []
total_detections = []
total_tracks = []

for tracker_instance in tracker:

    detector = tracker_instance.detector

    groundtruth_paths = set()
    detections = set() 
    tracks = set()
    
    for time, ctracks in tracker_instance.tracks_gen():
        detections |= detector.detections
        groundtruth_paths |= detector.groundtruth.groundtruth_paths
        tracks.update(ctracks)
        
    total_groundtruth_paths.append(groundtruth_paths)
    total_detections.append(detections)
    total_tracks.append(tracks)

    print("Tracker processed.")

Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
Tracker processed.
